In [ ]:
from util import get_spec, DotDict, get_generalization, camel_to_snake, snake_to_camel
from os import path, remove
from textwrap import wrap, indent
from networkx import DiGraph, topological_sort, simple_cycles
from re import compile as re_compile

BASE_DIR = filename = '../django-xmi/models/'
FIELD_MAPPINGS = dict(boolean=dict(name='BooleanField', attrs=[]),
                      string=dict(name='CharField', attrs=['max_length=255']))

RESERVED_TERMS = __builtin__.__dir__() + ['False', 'class', 'finally', 'is', 'return', 'None', 'continue',
                                          'for', 'lambda', 'try', 'True', 'def', 'from', 'nonlocal', 'while',
                                          'and', 'del', 'global', 'not', 'with', 'as', 'elif', 'if', 'or',
                                          'yield', 'assert', 'else', 'import', 'pass', 'break', 'except',
                                          'in', 'raise']
RESERVED_TERMS = tuple(term for term in RESERVED_TERMS if '_' != term[0])
INDENT = ' ' * 4

In [ ]:
uml = get_spec('http://www.omg.org/spec/UML/20131001/UML.xmi')

In [ ]:
sysml = get_spec('http://www.omg.org/spec/SysML/20150709/SysML.xmi')

# Parse elements into a `DotDict`

In [ ]:
elements = DotDict({})
profiles = {'UML': uml.XMI.Package.packagedElement,
            'SysML': sysml.XMI.Profile.packagedElement}
ignore = re_compile('^[aeAE]_')
ascii_fix = re_compile(r'[^\x00-\x7F]+')

for profile_name, profile in profiles.items():
    for package_name, package in profile.items():
        for elem_name, element in package.packagedElement.items():
            element.update({'__package__': package_name,
                            '__profile__': profile_name,
                            '__ignore__': bool(ignore.match(elem_name)),
                            '__modelclass__': get_generalization(element) or 'models.Model',
                            '__docstring__': element.get('ownedComment', {}).get('body', ''),
                            '__is_abstract__': element.get('isAbstract', False)})
            element.__docstring__ = ascii_fix.sub("'", element.__docstring__)
            for attr_name in element.get('ownedAttribute', {}).keys():
                attr = element['ownedAttribute'][attr_name]
                if 'ownedComment' in attr:
                    attr['help_text'] = ascii_fix.sub("'", attr['ownedComment'].get('body', ''))
            elements[camel_to_snake(element.name)] = element

In [ ]:
dependency_graph = DiGraph()
BASE_TYPES = ('element',)

for elem_name, element in elements.items():
    if element.__ignore__:
        continue

    if elem_name in BASE_TYPES:
        element.__modelclass__ = 'models.Model'

    if ',' in element.__modelclass__:
        for dependency in element.__modelclass__.split(', '):
            dependency_graph.add_edge(elem_name, camel_to_snake(dependency))
    else:
        dependency_graph.add_edge(elem_name, camel_to_snake(element.__modelclass__))

sorted_elements = list(topological_sort(dependency_graph))[:-1]
sorted_elements.reverse()

# Declare Helper Functions

In [ ]:
def write_literals(literals, file, element_name):
    choices = []
    i = -1
    use_descriptions = all('ownedComment' in lit for lit in literals.values())
    for term, literal in literals.items():
        i += 1
        value = literal.ownedComment.body if use_descriptions else term
        code = "'{}'".format(term.lower()) if use_descriptions else i
        
        file.write(INDENT + '{constant} = {code}\n'.format(constant=term.upper(), code=code))
        choices.append("({code}, '{value}'),\n".format(code=term.upper(), value=value))
    
    if i >= 0:
        field = 'CharField(max_length=255, ' if use_descriptions else 'IntegerField('
        file.write(INDENT + 'CHOICES = (\n')
        file.write(INDENT*2 + (INDENT*2).join(choices))
        file.write(INDENT + ')\n\n')

        field_str = INDENT + '{element_name} = models.{field}choices=CHOICES, default={default})\n'
        file.write(field_str.format(element_name=element_name,
                                      field=field,
                                      default=term.upper()))
        return True
    else:
        return False

In [ ]:
def write_attributes(attributes, file):
    """Write the ownedAttributes to the Django models file."""
    
    # TODO: this should probably be somewhere else
    if all(subattr in attributes for subattr in ('id', 'name', 'type')):
        element['ownedAttribute'] = DotDict({})
        element['ownedAttribute'][attributes.name] = DotDict(attributes)
        attributes = element.get('ownedAttribute', {})
        
    found_attributes = False
    redefined = {}
    
    """def __init__(self, *args, **kwargs):
    if 'value' not in kwargs:
        kwargs['value'] = 9
    super(Bar, self).__init__(*args, **kwargs)
    """
    for attr_name in sorted(attributes.keys()):
        
        attr = attributes[attr_name]
        attr_name = camel_to_snake(attr_name)
        if attr_name in RESERVED_TERMS:
            attr_name += '_'  # as per PEP-8's recommendation
        attr.name = attr_name
        
        # If the model is redefining an attribute in a superclass,
        # we need to handle it differently due to Django's limitations
        if 'redefinedProperty' in attr:
            redefined[attr.name] = attr
            continue

        subattrs = []

        if isinstance(attr.type, str):
            attr.__field__ = 'ForeignKey'
            attr.__other__ = attr.type
        elif isinstance(attr.type, dict):
            if 'idref' in attr.type:
                attr.__field__ = 'ForeignKey'
                attr.__other__ = attr.type['idref'].split('_')[-1]
            elif 'href' in attr.type:
                href = attr.type['href'].split('#')[-1].lower()
                if href in FIELD_MAPPINGS:
                    attr.__field__ = FIELD_MAPPINGS[href]['name']
                    subattrs += FIELD_MAPPINGS[href]['attrs']
                else:
                    attr.__field__ = 'ForeignKey'
                    attr.__other__ = attr.type['href'].split('#')[-1]

        if attr.get('__other__', None):
            if attr.__other__ == element.name:
                attr.__other__ = 'self'
            subattrs.append("'{}'".format(attr.__other__))

        help_text = attr.get('help_text', None)
        if help_text:
            quotation = '"""' if '"' in help_text else '"'
            subattrs += ['help_text={}{}{}'.format(quotation, help_text, quotation)]

        attr.__subattrs__ = ', '.join(subattrs)

        file.write('    {name} = models.{__field__}({__subattrs__})\n'.format(**attr))
        
        found_attributes = True
        
    if redefined:
        file.write('\n' + INDENT + 'def __init__(self, *args, **kwargs):\n')
        for name, redefine in redefined.items():
            if 'defaultValue' in redefine and 'instance' in redefine.defaultValue:
                default = redefine.defaultValue.instance.split('-')[-1]
                file.write(INDENT * 2 + "if '{}' not in kwargs:\n".format(name))
                file.write(INDENT * 3 + "kwargs['{}'] = '{}'\n".format(name, default))
            else:
                print("Could not redefine {}!".format(name))
                
        file.write(INDENT * 2 + 'super({name}).__init__(*args, **kwargs)\n'.format(**element))
        

    return found_attributes

In [ ]:
def write_operations(operations, file, element):
    if not operations:
        return False
    
    # TODO: this should probably be somewhere else
    if all(subattr in operations for subattr in ('id', 'name', 'type')):
        element['ownedOperation'] = DotDict({})
        element['ownedOperation'][operations.name] = DotDict(operations)
        operations = element.get('ownedOperation', {})
    
    for op in operations.values():
        file.write('\n')
        
        name = op.name
        if name in element.get('ownedAttribute', {}):
            name += '_operation'
            
        file.write(INDENT + 'def {}(self):\n'.format(camel_to_snake(name)))
        if 'ownedComment' in op:
            file.write(INDENT * 2 + '"""\n')
            for line in [indent(s, ' ' * 4) for s in wrap('{}\n'.format(op.ownedComment.body), 80)]:
                file.write(INDENT + line + '\n')
            file.write(INDENT * 2 + '"""\n')
        ocl = op.get('ownedRule', {}).get('specification', {}).get('body', '')
        if ocl:
            file.write(INDENT * 2 + '# OCL:\n')
            if '\n' in ocl:
                ocl = ocl.replace('\n\n', '\n').replace('\n', '\n' + INDENT * 3)
                file.write(INDENT * 2 + '"""    {}"""\n'.format(ocl))
            else:
                file.write(INDENT * 2 + '#     {}\n'.format(ocl))
        file.write(INDENT * 2 + 'pass\n')
        return True

In [ ]:
def write_meta(element, file):
    found_meta = False
    if element.__is_abstract__:
        file.write('\n' + INDENT + 'class Meta:\n' + INDENT * 2 + 'abstract = True\n')
        found_meta = True
    return found_meta

# Write Django models to files

In [ ]:
loaded = []
for profile in ('uml', 'sysml'):
    filename = path.join(BASE_DIR, "{}.py".format(profile))
    if path.exists(filename):
        remove(filename)
    with open(filename, 'w') as file:
        file.write('from django.db import models\n')
        for other in loaded:
            file.write('from .{} import *\n'.format(other))
    loaded.append(profile)
        
for element_name in sorted_elements:
    if element_name not in elements:
        print('could not find "{}"'.format(element_name))
        continue
    element = elements[element_name]
    
    filename = path.join(BASE_DIR, "{}.py".format(element.__profile__))
    
    with open(filename, 'a') as file:
        file.write('\n\n')
        file.write('class {name}({__modelclass__}):\n'.format(**element))
        if element.__docstring__:
            file.write(INDENT + '"""\n')
            for line in [indent(s, ' ' * 4) for s in wrap('{__docstring__}\n'.format(**element), 80)]:
                file.write(line + '\n')
            file.write(INDENT + '"""\n')
        file.write('\n')
            
        found_literals = write_attributes(element.get('ownedAttribute', {}), file)
        found_attributes = write_literals(element.get('ownedLiteral', {}), file, element_name)
        found_meta = write_meta(element, file)
        found_operations = write_operations(element.get('ownedOperation', {}), file, element)
        
        if not any((found_literals,
                    found_attributes,
                    found_operations,
                    found_meta)):
            file.write(INDENT + 'pass\n')

# Sandbox

In [ ]:
elements.packageable_element.ownedAttribute.visibility.redefinedProperty

In [ ]:
elements.named_element.ownedAttribute.visibility

In [ ]:
vis = elements.packageable_element.ownedAttribute.get('visibility')

In [ ]:
vis.defaultValue

In [ ]:
elements.visibility_kind.name

In [ ]:
owned = set()
for elem in elements.values():
    owned = owned.union(set([key for key in elem.keys() if 'owned' in key.lower()]))
{k: k.replace('owned', '').lower() + 's' for k in owned if k[:5] == 'owned'}

In [ ]:
{k for k,v in elements.items() if 'ownedRule' in v.keys()}

In [ ]:
elements.element.ownedOperation.allOwnedElements.ownedParameter.lowerValue

In [ ]:
elements.visibility_kind.ownedLiteral.package.ownedComment.body

In [ ]:
keys = set()
for elem in elements.values():
    keys = keys.union(set(elem.keys()))
keys